In [1]:
from rdkit import rdBase, Chem
from rdkit.Chem import AllChem, Draw
from rdkit.Chem.Draw import IPythonConsole
import sys, py3Dmol
import pandas as pd
from mordred import Calculator,descriptors
import numpy as np
import math
# データ可視化ライブラリ
import matplotlib.pyplot as plt

In [29]:
df = pd.DataFrame(pd.read_csv('DataTg.csv'))
#SMILESキーの入った列を指定
smiles = df["dataOriginal...1."]

In [3]:
mols = [Chem.MolFromSmiles(smile) for smile in smiles]

In [7]:
mols_from_sm = mols

#3次元構造発生用の関数を定義（力場が複数あります）
def DGuff(mols):
    DG_uff = []
    for mol in mols:
        mh = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mh, useBasicKnowledge=False, useExpTorsionAnglePrefs=False)
        if AllChem.UFFHasAllMoleculeParams(mh):
            AllChem.UFFOptimizeMolecule(mh)
            DG_uff.append(mh)
    return DG_uff

def DGmmff(mols):
    DG_mmff = []
    for mol in mols:
        mh = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mh, useBasicKnowledge=False, useExpTorsionAnglePrefs=False)
        if AllChem.MMFFHasAllMoleculeParams(mh):
            AllChem.MMFFOptimizeMolecule(mh)
            DG_mmff.append(mh)
    return DG_mmff

def ETDG(mols):
    ETDG_mols = []
    for mol in mols:
        mh = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mh, AllChem.ETDG())
        ETDG_mols.append(mh)
    return ETDG_mols

def KDG(mols):
    KDG_mols = []
    for mol in mols:
        mh = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mh, AllChem.KDG())
        KDG_mols.append(mh)
    return KDG_mols


def ETKDG(mols, version=1):
    ETKDG_mols = []
    for mol in mols:
        mh = Chem.AddHs(mol)
        if version == 1:
            p = AllChem.ETKDG()
        elif version == 2:
            p = AllChem.ETKDGv2()
        else:
            print('invalid input')
        AllChem.EmbedMolecule(mh, p)
        ETKDG_mols.append(mh)
    return ETKDG_mols
 
DG_m = DGuff(mols_from_sm)
ETDG_m = ETDG(mols_from_sm)
ETKDGv1_m = ETKDG(mols_from_sm)
ETKDGv2_m = ETKDG(mols_from_sm, 2)

In [8]:
#構造発生回数の設定（分子力学による3次元構造は、分布を持つため）
iteration_num = 50
set_num = list(range(iteration_num))
rows_num =list(range(len(DG_m)))

In [9]:
#3D記述子のみを計算する関数を設定
calc_dummy = Calculator(descriptors, ignore_3D=False)
my_desc_names = ["PNSA1","PNSA2","PNSA3","PNSA4","PNSA5","PPSA1","PPSA2","PPSA3","PPSA4","PPSA5","DPSA1","DPSA2","DPSA3","DPSA4","DPSA5","FNSA1","FNSA2","FNSA3","FNSA4","FNSA5","FPSA1","FPSA2","FPSA3","FPSA4","FPSA5","WNSA1","WNSA2","WNSA3","WNSA4","WNSA5","WPSA1","WPSA2","WPSA3","WPSA4","WPSA5","RNCS","RPCS","TASA","TPSA","RASA","RPSA","GeomDiameter","GeomRadius","GeomShapeIndex","GeomPetitjeanIndex","GRAV","GRAVH","GRAVp","GRAVHp","Mor01","Mor02","Mor03","Mor04","Mor05","Mor06","Mor07","Mor08","Mor09","Mor10","Mor11","Mor12","Mor13","Mor14","Mor15","Mor16","Mor17","Mor18","Mor19","Mor20","Mor21","Mor22","Mor23","Mor24","Mor25","Mor26","Mor27","Mor28","Mor29","Mor30","Mor31","Mor32","Mor01m","Mor02m","Mor03m","Mor04m","Mor05m","Mor06m","Mor07m","Mor08m","Mor09m","Mor10m","Mor11m","Mor12m","Mor13m","Mor14m","Mor15m","Mor16m","Mor17m","Mor18m","Mor19m","Mor20m","Mor21m","Mor22m","Mor23m","Mor24m","Mor25m","Mor26m","Mor27m","Mor28m","Mor29m","Mor30m","Mor31m","Mor32m","Mor01v","Mor02v","Mor03v","Mor04v","Mor05v","Mor06v","Mor07v","Mor08v","Mor09v","Mor10v","Mor11v","Mor12v","Mor13v","Mor14v","Mor15v","Mor16v","Mor17v","Mor18v","Mor19v","Mor20v","Mor21v","Mor22v","Mor23v","Mor24v","Mor25v","Mor26v","Mor27v","Mor28v","Mor29v","Mor30v","Mor31v","Mor32v","Mor01se","Mor02se","Mor03se","Mor04se","Mor05se","Mor06se","Mor07se","Mor08se","Mor09se","Mor10se","Mor11se","Mor12se","Mor13se","Mor14se","Mor15se","Mor16se","Mor17se","Mor18se","Mor19se","Mor20se","Mor21se","Mor22se","Mor23se","Mor24se","Mor25se","Mor26se","Mor27se","Mor28se","Mor29se","Mor30se","Mor31se","Mor32se","Mor01p","Mor02p","Mor03p","Mor04p","Mor05p","Mor06p","Mor07p","Mor08p","Mor09p","Mor10p","Mor11p","Mor12p","Mor13p","Mor14p","Mor15p","Mor16p","Mor17p","Mor18p","Mor19p","Mor20p","Mor21p","Mor22p","Mor23p","Mor24p","Mor25p","Mor26p","Mor27p","Mor28p","Mor29p","Mor30p","Mor31p","Mor32p","MOMI-X","MOMI-Y","MOMI-Z","PBF"]
my_descs = []
for i, desc in enumerate(calc_dummy.descriptors):
    if desc.__str__()  in my_desc_names:
       my_descs.append(desc)

calc_real = Calculator(my_descs, ignore_3D=False)

In [10]:
#各力場により、指定された回数だけ構造を発生、記述子計算
descriptors = {}

for i in set_num:
    descriptors[i] = pd.DataFrame(calc_real.pandas(ETKDG(mols_from_sm, 2)))
    detkdg = pd.DataFrame(calc_real.pandas(ETKDG(mols_from_sm)))
    descriptors[i] = pd.concat([descriptors[i], detkdg.add_suffix("_ETKDG")], axis = 1)
    detdg = pd.DataFrame(calc_real.pandas(ETDG(mols_from_sm)))
    descriptors[i] = pd.concat([descriptors[i], detdg.add_suffix("_ETDG")], axis = 1)
    dkdg = pd.DataFrame(calc_real.pandas(KDG(mols_from_sm)))
    descriptors[i] = pd.concat([descriptors[i], dkdg.add_suffix("_KDG")], axis = 1)
    ddguff = pd.DataFrame(calc_real.pandas(DGuff(mols_from_sm)))
    descriptors[i] = pd.concat([descriptors[i], ddguff.add_suffix("_DGuff")], axis = 1)
    ddgmmff = pd.DataFrame(calc_real.pandas(DGmmff(mols_from_sm)))
    descriptors[i] = pd.concat([descriptors[i], ddgmmff.add_suffix("_DGmmff")], axis = 1)


100%|██████████| 47/47 [00:00<00:00, 157.77it/s]


100%|██████████| 47/47 [00:00<00:00, 157.46it/s]


In [11]:
descriptors[2] #2回目の構造発生＋記述子計算結果

,PNSA1,PNSA2,PNSA3,PNSA4,PNSA5,PPSA1,PPSA2,PPSA3,PPSA4,PPSA5,...,Mor27p_DGmmff,Mor28p_DGmmff,Mor29p_DGmmff,Mor30p_DGmmff,Mor31p_DGmmff,Mor32p_DGmmff,MOMI-X_DGmmff,MOMI-Y_DGmmff,MOMI-Z_DGmmff,PBF_DGmmff
0,72.2876,-15.3699,-7.68497,-2.56166,-7.68497,91.4454,19.4433,4.86083,3.24055,4.86083,...,-0.037007,-0.060049,-0.037463,0.003764,0.062930,0.047340,20.280741,16.793785,3.486957,1.475910e-07
1,88.7714,-74.0676,-16.4228,-4.93784,-18.5169,186.924,155.963,12.1606,10.3975,14.1784,...,-0.027244,-0.049844,-0.120077,0.037530,0.104089,0.064084,346.387581,245.628846,107.227458,2.388346e-01
2,126.264,-23.2626,-10.0736,-3.8771,-7.7542,63.9749,11.7866,3.95366,1.96443,3.92887,...,-0.005054,-0.024003,-0.063564,0.001647,0.064704,0.056469,92.187767,83.334764,8.853003,5.225558e-07
3,missing 3D coordinate (PNSA1/AtomicSurfaceArea),missing 3D coordinate (PNSA1/AtomicSurfaceArea),missing 3D coordinate (PNSA1/AtomicSurfaceArea),missing 3D coordinate (PNSA1/AtomicSurfaceArea),missing 3D coordinate (PNSA1/AtomicSurfaceArea),missing 3D coordinate (PNSA1/AtomicSurfaceArea),missing 3D coordinate (PNSA1/AtomicSurfaceArea),missing 3D coordinate (PNSA1/AtomicSurfaceArea),missing 3D coordinate (PNSA1/AtomicSurfaceArea),missing 3D coordinate (PNSA1/AtomicSurfaceArea),...,-0.157172,0.049825,-0.018340,0.038763,0.065297,-0.142273,880.203954,707.441581,416.933926,9.106699e-01
4,129.834,-62.6474,-8.86714,-3.91546,-7.83093,149.275,72.0279,9.02467,4.50174,9.00349,...,-0.144846,-0.040453,-0.005500,0.016877,0.049484,-0.071843,418.797171,326.539975,98.822009,2.232359e-01
5,65.3187,-16.0358,-5.58943,-1.78175,-5.34526,129.824,31.8719,5.3136,3.54132,5.31199,...,0.006077,-0.092178,-0.072065,0.002957,0.117205,0.055933,61.727148,53.921611,10.979272,1.971577e-01
6,66.9251,-18.2713,-4.80261,-1.52261,-4.56783,155.19,42.3687,5.26389,3.53072,5.29608,...,0.040521,-0.119369,-0.117777,0.007013,0.188330,0.042016,109.173708,60.267651,55.276552,2.962723e-01
7,52.4885,-21.5109,-14.322,-2.15109,-10.7555,153.906,63.074,7.49161,6.3074,7.88425,...,0.022095,-0.003686,-0.119527,-0.012584,0.092848,0.065143,84.358719,75.307797,28.163969,4.242791e-01
8,82.7354,-62.1314,-19.0977,-5.17762,-20.7105,167.854,126.053,11.0038,10.5044,14.0058,...,-0.021858,-0.069890,-0.083880,0.026810,0.079727,0.062205,253.399072,195.506676,61.150752,1.498333e-01
9,84.3108,-67.8428,-18.2874,-5.65357,-22.6143,162.891,131.075,12.3639,10.9229,14.5638,...,-0.032996,-0.060547,-0.051132,-0.029970,0.119270,0.055653,250.394511,178.687033,74.979542,1.501416e-01


In [12]:
#出来上がった記述子の統計値をとります
distribution = {}

for m in rows_num:

    numbers = pd.DataFrame()
    for n in set_num:
        number = pd.Series(descriptors[n].iloc[m])
        numbers = numbers.append(number, ignore_index=True)
    distribution[m] = numbers.describe()[3:]


In [14]:
#2つめの分子に関する記述子の分布
distribution[2]

,DPSA1,DPSA1_DGmmff,DPSA1_DGuff,DPSA1_ETDG,DPSA1_ETKDG,DPSA1_KDG,DPSA2,DPSA2_DGmmff,DPSA2_DGuff,DPSA2_ETDG,...,WPSA4_DGuff,WPSA4_ETDG,WPSA4_ETKDG,WPSA4_KDG,WPSA5,WPSA5_DGmmff,WPSA5_DGuff,WPSA5_ETDG,WPSA5_ETKDG,WPSA5_KDG
min,-64.624902,-60.648274,-61.824982,-63.440334,-63.250763,-59.241625,34.876767,34.961395,35.039000,35.085117,...,0.379345,0.374544,0.373231,0.385470,0.741552,0.762918,0.758690,0.749088,0.746462,0.770941
25%,-61.960585,-59.911854,-61.207661,-62.631875,-61.988338,-55.748471,35.105294,35.128090,35.269989,35.267801,...,0.383217,0.379263,0.377778,0.398267,0.752555,0.768827,0.766434,0.758527,0.755556,0.796534
50%,-61.224688,-59.686616,-60.900984,-61.693219,-61.220167,-55.122669,35.194831,35.197230,35.305423,35.310058,...,0.384223,0.383767,0.380807,0.401521,0.760441,0.770468,0.768446,0.767534,0.761615,0.803042
75%,-60.366565,-59.474921,-60.674039,-60.381402,-60.234705,-53.993072,35.266860,35.250042,35.338332,35.381347,...,0.385238,0.386094,0.383175,0.404351,0.767460,0.772358,0.770475,0.772189,0.766351,0.808703
max,-58.461659,-58.831939,-60.224368,-57.750179,-59.454414,-52.664966,35.442379,35.420578,35.393092,35.482477,...,0.387404,0.396816,0.388189,0.413382,0.780354,0.778356,0.774808,0.793632,0.776379,0.826765


In [15]:
#分布に関する情報を含め、全てを記述子とすべく列名のリストを作成します（記述子数×力場数×分布情報個）
dist_sample = distribution[1]
rows_num_2 =list(range(len(dist_sample)))
index_num = list(range(len(dist_sample.T)))
nameslist = []

for cname in index_num:
    for rname in rows_num_2:
        namescolumn = (dist_sample.index[rname] + dist_sample.columns[cname])
        nameslist.append(namescolumn)


In [16]:
nameslist

['minDPSA1',
 '25%DPSA1',
 '50%DPSA1',
 '75%DPSA1',
 'maxDPSA1',
 'minDPSA1_DGmmff',
 '25%DPSA1_DGmmff',
 '50%DPSA1_DGmmff',
 '75%DPSA1_DGmmff',
 'maxDPSA1_DGmmff',
 'minDPSA1_DGuff',
 '25%DPSA1_DGuff',
 '50%DPSA1_DGuff',
 '75%DPSA1_DGuff',
 'maxDPSA1_DGuff',
 'minDPSA1_ETDG',
 '25%DPSA1_ETDG',
 '50%DPSA1_ETDG',
 '75%DPSA1_ETDG',
 'maxDPSA1_ETDG',
 'minDPSA1_ETKDG',
 '25%DPSA1_ETKDG',
 '50%DPSA1_ETKDG',
 '75%DPSA1_ETKDG',
 'maxDPSA1_ETKDG',
 'minDPSA1_KDG',
 '25%DPSA1_KDG',
 '50%DPSA1_KDG',
 '75%DPSA1_KDG',
 'maxDPSA1_KDG',
 'minDPSA2',
 '25%DPSA2',
 '50%DPSA2',
 '75%DPSA2',
 'maxDPSA2',
 'minDPSA2_DGmmff',
 '25%DPSA2_DGmmff',
 '50%DPSA2_DGmmff',
 '75%DPSA2_DGmmff',
 'maxDPSA2_DGmmff',
 'minDPSA2_DGuff',
 '25%DPSA2_DGuff',
 '50%DPSA2_DGuff',
 '75%DPSA2_DGuff',
 'maxDPSA2_DGuff',
 'minDPSA2_ETDG',
 '25%DPSA2_ETDG',
 '50%DPSA2_ETDG',
 '75%DPSA2_ETDG',
 'maxDPSA2_ETDG',
 'minDPSA2_ETKDG',
 '25%DPSA2_ETKDG',
 '50%DPSA2_ETKDG',
 '75%DPSA2_ETKDG',
 'maxDPSA2_ETKDG',
 'minDPSA2_KDG',
 '25%DPS

In [17]:
#各分子に対し、全ての記述子を整理して表示
desc_dist = pd.DataFrame()

for m in rows_num:

    numbers = pd.DataFrame()
    for n in set_num:
        number = pd.Series(descriptors[n].iloc[m])
        numbers = numbers.append(number, ignore_index=True)
    em = numbers.describe()[3:]
    em = pd.DataFrame(em.values.reshape(1, len(em)*len(em.T), order = 'F'))
    desc_dist = desc_dist.append(em, ignore_index = True)


In [18]:
desc_dist.columns = [nameslist]

In [28]:
#最終的な記述子の計算結果（3Dだけで6000以上あります）
desc_dist.head()

,minDPSA1,25%DPSA1,50%DPSA1,75%DPSA1,maxDPSA1,minDPSA1_DGmmff,25%DPSA1_DGmmff,50%DPSA1_DGmmff,75%DPSA1_DGmmff,maxDPSA1_DGmmff,...,minWPSA5_ETKDG,25%WPSA5_ETKDG,50%WPSA5_ETKDG,75%WPSA5_ETKDG,maxWPSA5_ETKDG,minWPSA5_KDG,25%WPSA5_KDG,50%WPSA5_KDG,75%WPSA5_KDG,maxWPSA5_KDG
0,18.724974,20.218300,21.214784,21.890085,23.828661,20.804865,21.897085,22.282741,22.706230,23.767205,...,0.770938,0.781842,0.788927,0.795131,0.808613,0.769782,0.783480,0.791619,0.803717,0.814879
1,88.497927,91.945827,93.530543,95.280608,98.153119,91.292615,92.499154,92.902593,93.389552,94.757196,...,3.731590,3.804456,3.843716,3.878132,3.957060,3.793989,3.942065,4.000236,4.073694,4.198193
2,-64.624902,-61.960585,-61.224688,-60.366565,-58.461659,-60.648274,-59.911854,-59.686616,-59.474921,-58.831939,...,0.746462,0.755556,0.761615,0.766351,0.776379,0.770941,0.796534,0.803042,0.808703,0.826765
3,-24.631658,-23.141817,9.240122,10.204323,10.780084,-25.888754,-25.083464,-24.372971,5.839622,6.745511,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18.814502,20.510317,21.619443,22.827008,24.771458,30.893188,32.378082,32.794552,33.113511,33.752136,...,2.497566,2.522869,2.531381,2.544382,2.587308,2.585985,2.758726,2.823818,2.887442,2.975955


In [20]:
#2D記述子を計算
from mordred import Calculator, descriptors

mols_2d = mols_from_sm
calc = Calculator(descriptors, ignore_3D=True)
df_descriptors_mordred_2d = calc.pandas(mols_2d)


100%|██████████| 47/47 [00:00<00:00, 58.47it/s]


In [25]:
df_summary_3d = pd.concat([df_descriptors_mordred_2d, desc_dist],axis=1)

In [26]:
#NAを入力してからcsvファイルを出力
df_summary = df_summary_3d.astype(str)
masks_3d = df_summary.apply(lambda d: d.str.contains('[a-zA-Z]' ,na=True))
df_summary = df_summary[~masks_3d]
df_summary = df_summary.astype(float)
df_summary = df_summary.fillna("NA")

df_summary = pd.concat([df, df_summary], axis = 1)

df_summary.to_csv('DataTg_with2d_3d_dist.csv', index = False)